This code is for analyzing CSV files created by FIBSI. 

Import packages needed.

In [ ]:
import os
import pandas as pd
import numpy as np  # Import numpy for NaN handling

The following code calculates if a file has DSFs and the frequency of DSFs 

In [ ]:
# Directory where CSV files are located
directory = "HS23_DSFs_rest"

# Initialize lists to store data
filenames = []
dsf_percentages = []
counts_zero = []

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        
        # Initialize DSF% to 0 and count of '0' values to 0 for the current file
        dsf_percentage = 0
        count_zero = 0
        
        # Read the CSV file
        try:
            df = pd.read_csv(filepath)
            
            # Count '0' values in 'indicator' column
            count_zero = df[df['indicator'] == 0]['indicator'].count()
            
            # Determine DSF% based on count of '0' values
            if count_zero > 0:
                dsf_percentage = 1
            
        except pd.errors.EmptyDataError:
            # Handle empty file (no data)
            pass
        
        # Append data to lists
        filenames.append(filename)
        dsf_percentages.append(dsf_percentage)
        counts_zero.append(count_zero)

# Create a DataFrame
df_results = pd.DataFrame({
    'File': filenames,
    'DSF%': dsf_percentages,
    'Count of 0': counts_zero
})

# Print the DataFrame
print(df_results)

Save as CSV file. 

In [ ]:
# Save the DataFrame to a CSV file
output_csv = "HS23_DSF_count.csv"
df_results.to_csv(output_csv, index=False)

print(f"Results saved to {output_csv}")

This code calculates DSF amplitudes and puts the results in a dataframe.

In [ ]:
# Directory where CSV files are located
directory = "HS23_DSFs_rest"

# Initialize a dictionary to store original amplitudes
original_amplitudes_dict = {}

# Initialize a list to track maximum length of 'original amplitude' lists
max_length = 0

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        
        # Initialize DSF% to 0 and count of '0' values to 0 for the current file
        dsf_percentage = 0
        count_zero = 0
        original_amplitudes = []
        
        # Read the CSV file
        try:
            df = pd.read_csv(filepath)
            
            # Count '0' values in 'indicator' column
            count_zero = df[df['indicator'] == 0]['indicator'].count()
            
            # Extract 'original amplitude' values where 'indicator' is '0'
            original_amplitudes = df.loc[df['indicator'] == 0, 'original amplitude'].tolist()
            
            # Determine DSF% based on count of '0' values
            if count_zero > 0:
                dsf_percentage = 1
            
            # Store original amplitudes in dictionary with filename as key
            original_amplitudes_dict[filename] = original_amplitudes
            
            # Update max_length if current list length is greater
            max_length = max(max_length, len(original_amplitudes))
        
        except pd.errors.EmptyDataError:
            # Handle empty file (no data)
            original_amplitudes_dict[filename] = []

# Fill in missing values with NaN to ensure all lists are of the same length
for filename, amps in original_amplitudes_dict.items():
    if len(amps) < max_length:
        original_amplitudes_dict[filename].extend([np.nan] * (max_length - len(amps)))

# Create a DataFrame from the dictionary
df_original_amplitudes = pd.DataFrame(original_amplitudes_dict)

In [ ]:
# Save the DataFrame to a CSV file
output_csv = "HS23_DSF_amplitudes.csv"
df_original_amplitudes.to_csv(output_csv, index=False)

This code calculates DSF durations and puts them into a dataframe. and saves a CSV file. 

In [ ]:
# Directory where CSV files are located
directory = "HS23_DSFs_rest"

# Initialize a dictionary to store event durations
event_durations_dict = {}

# Initialize a variable to track the maximum length of event duration lists
max_length = 0

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        
        # Initialize event durations list for the current file
        event_durations = []
        
        # Read the CSV file
        try:
            df = pd.read_csv(filepath)
            
            # Filter rows where 'indicator' is '0'
            df_filtered = df[df['indicator'] == 0]
            
            # Calculate event durations
            durations = (pd.to_datetime(df_filtered['end time']) - pd.to_datetime(df_filtered['start time'])).dt.total_seconds()
            event_durations = durations.tolist()
            
            # Store event durations in dictionary with filename as key
            event_durations_dict[filename] = event_durations
            
            # Update max_length if current list length is greater
            max_length = max(max_length, len(event_durations))
        
        except pd.errors.EmptyDataError:
            # Handle empty file (no data)
            event_durations_dict[filename] = []

# Fill in missing values with NaN to ensure all lists are of the same length
for filename, durations in event_durations_dict.items():
    if len(durations) < max_length:
        event_durations_dict[filename].extend([np.nan] * (max_length - len(durations)))

# Create a DataFrame from the dictionary
df_event_durations = pd.DataFrame(event_durations_dict)

# Save the DataFrame to a new CSV file
df_event_durations.to_csv('HS23_DSF_durations.csv', index=False)

print("Event durations have been calculated and saved successfully.")
